# One At the Time (OAT) Sensitivity analysis

This is the simplest case. Also called a _local_ Sensitivity analysis. One parameter is changed by keeping all the other constant and the difference in results is compared to the change. This allows to investigate how much results are affected by the specific change in the parameter. This is good for some types of analysis, but has some problems. Main issue is that the effect of a change in the parameter might be different when other parameters assume different values...so OAT can be misleading!  This problem can only be solved with a global sensitivity analysis (next notebook)

This notebook show how to perform a simple One A Time (OAT) sensitivity analysis for a example product system of a biobased product, and calculated sensitivity ratios for each parameter.

In [1]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
bd.projects.set_current('advlca23') # Still working in the same project
bd.databases

Databases dictionary with 9 object(s):
	ALIGNED-biob-prod-dummy
	SAtestdb
	biosphere3
	ecoinvent 3.9.1 conseq
	exldb
	gsa_db
	sa_db
	testbiosphere
	testdb

We start by importing data about a fictional ("dummy") product system for a biobased product.

The product system includes different activities such as the production, use, and end of life of the biobased product.

In [3]:
# Import the dummy product system

# import data from csv
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb = mydata.copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 308.62it/s]

Vacuuming database 


'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011


In [4]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'process',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 23611}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [5]:
# calculation of static LCA score
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print("The static Global Warming impact score is", LCA.score, bd.methods[mymethod]['unit'])

The static Global Warming impact score is 121.67148771458245 kg CO2-Eq


### Now perform sensitivity analysis

The procedure is in **two steps**: 

1) A simulation is performed where the initial parameter (value of each exchange) is increased by 10%. New results are calculated. 

2) Sensitivity rations are calculated for each parameter and then ranked. 


#### Step 1

Iteration through all parameters, change the technology matrix and recaltulate results.

In [6]:
# Iterate through all exchanges in the foreground system and change the value up by 10%
par_name = []
par_values = []
par_upper_values = []
score_values = []
OAT_values  = []

mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)


for act in bd.Database('ALIGNED-biob-prod-dummy'): # for all activities...
    
    for exc in act.exchanges(): # for all exchanges...
        
        LCA.lci()
        LCA.lcia()
        print("initial value", LCA.score)
        par_name.append((act['code'], exc['input'][1]))
        par_values.append(exc['amount'])
        par_upper_values.append(exc['amount'] * 1.1)
        score_values.append(LCA.score)
        
        if exc['type'] == "biosphere":
            col = LCA.activity_dict[bd.Database('ALIGNED-biob-prod-dummy').get(exc['output'][1])['id']] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[bd.Database('biosphere3').get(exc['input'][1])['id']] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = LCA.biosphere_matrix[row,col] * 1.1 # print the initial and final value (to be substituted)
        elif exc['type'] == "production":
            col = LCA.activity_dict[bd.Database('ALIGNED-biob-prod-dummy').get(exc['output'][1])['id']] # find column index of A matrix for the activity
            row = LCA.activity_dict[bd.Database('ALIGNED-biob-prod-dummy').get(exc['input'][1])['id']] # find row index of A matrix for the exchange
            LCA.technosphere_matrix[row,col] = LCA.technosphere_matrix[row,col] * 1.1
        elif exc['type'] == "technosphere":
            try:
                exc_input_code = bd.Database('ALIGNED-biob-prod-dummy').get(exc['input'][1])
                if exc_input_code:
                    col = LCA.activity_dict[bd.Database('ALIGNED-biob-prod-dummy').get(exc['output'][1])['id']]
                    row = LCA.activity_dict[exc_input_code['id']]
                    LCA.technosphere_matrix[row, col] = LCA.technosphere_matrix[row, col] * 1.1
            except:
                exc_input_code = bd.Database('ecoinvent 3.9.1 conseq').get(exc['input'][1])
                if exc_input_code:
                    col = LCA.activity_dict[bd.Database('ALIGNED-biob-prod-dummy').get(exc['output'][1])['id']]
                    row = LCA.activity_dict[exc_input_code['id']]
                    LCA.technosphere_matrix[row, col] = LCA.technosphere_matrix[row, col] * 1.1
            
        LCA.redo_lci() # uses the new A matrix
        LCA.lcia()
        OAT_values.append(LCA.score)
        print('end value', LCA.score)
        print("---")

initial value 121.67148771458245
end value 115.24991952039284
---
initial value 115.24991952039284
end value 119.91222100378864
---
initial value 119.91222100378864
end value 121.57839978018588
---
initial value 121.57839978018588
end value 121.67148771458243
---
initial value 121.67148771458243
end value 110.61044337689316
---
initial value 110.61044337689316
end value 117.0320115710827
---
initial value 117.0320115710827
end value 121.67055405093389
---
initial value 121.67055405093389
end value 121.67148771458247
---
initial value 121.67148771458247
end value 117.03294523473134
---
initial value 117.03294523473134
end value 117.12603316912792
---
initial value 117.12603316912792
end value 121.67148771458247
---
initial value 121.67148771458247
end value 120.00530893818521
---
initial value 120.00530893818521
end value 120.05185290538361
---
initial value 120.05185290538361
end value 121.67148771458247
---
initial value 121.67148771458247
end value 117.00918623118679
---
initial valu

Let's look at the results. 

Initial and increased ('upper') parameter values and initial and end values of the Global Warming Impact (GWI) scores. 

In [7]:
sr = pd.DataFrame([par_name, par_values, par_upper_values, score_values, OAT_values], 
                  index = ['par_name','par_initial', 'par_upper', 'GWI_initial', 'GWI_end']).T
sr.head()

,par_name,par_initial,par_upper,GWI_initial,GWI_end
0,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,121.671488,115.24992
1,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,115.24992,119.912221
2,"(a37d149a-6508-4563-8af6-e5a39b4176df, 403a5c3...",0.5,0.55,119.912221,121.5784
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, f4dc7d2...",0.5,0.55,121.5784,121.671488
4,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,121.671488,110.610443


#### Step 2

**Sensitivity ratios** (SR) are calculated using the formula for discrete distributions, see equation (2) in Bisinella et al. (2016) 

_Bisinella, V., Conradsen, K., Christensen, T.H., Astrup, T.F., 2016. A global approach for sparse representation of uncertainty in Life Cycle Assessments of waste management systems. International Journal of Life Cycle Assessment._ https://doi.org/10.1007/s11367-015-1014-4

In [8]:
# calcualte sensitivity ratios
sr['SR'] = ((sr['GWI_end']-sr['GWI_initial'])/sr['GWI_initial']) / ((sr['par_upper']-sr['par_initial'])/sr['par_initial']) 

sr['SR_abs'] = abs(sr['SR'])

In [9]:
sr.head()

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs
0,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,121.671488,115.24992,-0.527779,0.527779
1,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,115.24992,119.912221,0.404538,0.404538
2,"(a37d149a-6508-4563-8af6-e5a39b4176df, 403a5c3...",0.5,0.55,119.912221,121.5784,0.13895,0.13895
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, f4dc7d2...",0.5,0.55,121.5784,121.671488,0.007657,0.007657
4,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,121.671488,110.610443,-0.909091,0.909091


We can rank the parameters to see the most sensitive ones. I use the absolute value of SR in this case.

In [10]:
sr_sorted = sr.sort_values('SR_abs', ascending = False, ignore_index = True).copy() # important to re-index...
sr_sorted

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs
0,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,121.671488,110.610443,-0.909091,0.909091
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 97601e6...",0.5,0.55,117.009186,123.838953,0.583695,0.583695
2,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, a37d149...",50.0,55.0,110.610443,117.032012,0.580557,0.580557
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,121.671488,115.24992,-0.527779,0.527779
4,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,115.24992,119.912221,0.404538,0.404538
5,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, c8301e7...",50.0,55.0,117.032012,121.670554,0.396348,0.396348
6,"(c8301e73-d521-4a89-998b-30b7e7751011, 349b29d...",1.0,1.1,117.126033,121.671488,0.388082,0.388082
7,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,121.671488,117.009186,-0.383188,0.383188
8,"(c8301e73-d521-4a89-998b-30b7e7751011, c8301e7...",1.0,1.1,121.671488,117.032945,-0.381235,0.381235
9,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,123.944215,121.671488,-0.183367,0.183367


What are the most sensitive parameters in plain English?

In [11]:
#scroll through the table, identify the specific exchanges.
testing = []

for i in range(0, sr_sorted.shape[0]):
    
    par = sr_sorted.iloc[i,0] 
    
    for i in bd.Database('ALIGNED-biob-prod-dummy').get(par[0]).exchanges():
        if i['input'][1] == par[1]:
            print(i)
            testing.append(str(i))

Exchange: 1.0 year 'Biobased-product-use' (year, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 0.5 kilogram 'nutrient supply from NPK (26-15-15) fertiliser' (kilogram, RER, None) to 'Biomass-growth' (kilogram, None, None)>
Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 1.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-manufacturing' (kilogram, None, None)>
Exchange: 0.5 kilogram 'Biomass-growth' (kilogram, None, None) to 'Biobased-product-manufacturing' (kilogram, None, None)>
Exchange: 50.0 kilogram 'Biobased-product-eol' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 1.0 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'Biobased-product-eol' (kilogram, None, None)>
Exchange: 1.0 kilogram 'Biomass-growth' (kilogram, None, None) to 'Biomass-growth' (kilogram, None, None)>
Exchange: 1.0 kilog

In [12]:
sr_sorted['par_long-name'] = testing
sr_sorted

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs,par_long-name
0,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,121.671488,110.610443,-0.909091,0.909091,Exchange: 1.0 year 'Biobased-product-use' (yea...
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 97601e6...",0.5,0.55,117.009186,123.838953,0.583695,0.583695,Exchange: 0.5 kilogram 'nutrient supply from N...
2,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, a37d149...",50.0,55.0,110.610443,117.032012,0.580557,0.580557,Exchange: 50.0 kilogram 'Biobased-product-manu...
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,121.671488,115.24992,-0.527779,0.527779,Exchange: 1.0 kilogram 'Biobased-product-manuf...
4,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,115.24992,119.912221,0.404538,0.404538,Exchange: 0.5 kilogram 'Biomass-growth' (kilog...
5,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, c8301e7...",50.0,55.0,117.032012,121.670554,0.396348,0.396348,Exchange: 50.0 kilogram 'Biobased-product-eol'...
6,"(c8301e73-d521-4a89-998b-30b7e7751011, 349b29d...",1.0,1.1,117.126033,121.671488,0.388082,0.388082,"Exchange: 1.0 kilogram 'Carbon dioxide, fossil..."
7,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,121.671488,117.009186,-0.383188,0.383188,Exchange: 1.0 kilogram 'Biomass-growth' (kilog...
8,"(c8301e73-d521-4a89-998b-30b7e7751011, c8301e7...",1.0,1.1,121.671488,117.032945,-0.381235,0.381235,Exchange: 1.0 kilogram 'Biobased-product-eol' ...
9,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,123.944215,121.671488,-0.183367,0.183367,"Exchange: -1.0 kilogram 'Carbon dioxide, fossi..."
